# 데이터 불러오기
- SAheart.csv : 남아프리카 심장병 데이터

In [16]:
import pandas as pd
data = pd.read_csv("../Data/SAheart.csv", sep = ",", index_col = 0) # SAheart.csv 불러오기, , 로 구분해서 행 지정, 첫 컬럼을 Row로 지정

In [17]:
data.head() # 불러온 데이터 확인
# data.shape : 데이터 개수, 컬럼 개수 ; (462, 10)
# data.columns : 데이터 컬럼 명 확인 ; Index(['sbp', 'tobacco', 'ldl', 'adiposity', 'famhist', 'typea', 'obesity','alcohol', 'age', 'chd'],dtype='object')

,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
row.names,,,,,,,,,,
1,160,12.00,5.73,23.11,Present,49,25.30,97.20,52,1
2,144,0.01,4.41,28.61,Absent,55,28.87,2.06,63,1
3,118,0.08,3.48,32.28,Present,52,29.14,3.81,46,0
4,170,7.50,6.41,38.03,Present,51,31.99,24.26,58,1
5,134,13.60,3.50,27.78,Present,60,25.99,57.34,49,1


# 데이터 전처리
- 정규화 : 수치형 변수들의 단위가 서로 다르기 때문에, 규격을 맞추어주기 위한 방법
- 더미변수화 : 범주형 변수들의 정보를 반영하기 위해, [True, False], [False, True] 로 지정

In [18]:
# 정규화
from sklearn import preprocessing
data1 = data.iloc[:, :4]
data2 = data.iloc[:, 5:9]
data3 = pd.concat([data1, data2], axis=1)
scaler = preprocessing.MinMaxScaler()
# scaler = preprocessing.Normalizer()
# scaler = preprocessing.StandardScaler()
scaler.fit(data3) # 정규화 함수 매핑
data4 = scaler.transform(data3) # 정규화 함수 적용
scaled_data = pd.DataFrame(data4, columns = data3.columns, index = data3.index) # 정규화 적용된 데이터 저장, 컬럼명은 기존의 데이터 컬럼명 적용

In [19]:
# 더미변수화
scaled_data['famhist_true'] = data['famhist'] == 'Present'
scaled_data['famhist_false'] = data['famhist'] == 'Absent'

In [20]:
preprocessed_data = pd.concat([scaled_data, data['chd']], axis = 1) # 정규화, 더미변수, 종속변수를 통합한 데이터

# 데이터 분할
- 학습 데이터와 검증 데이터로 분할 (Train, Test)

In [21]:
from sklearn import model_selection
train, test = model_selection.train_test_split(preprocessed_data)
x_train = train.loc[:, train.columns != "chd"] # 학습 데이터 중 독립 변수
y_train = train['chd'] # 학습 데이터 중 종속 변수
x_test = test.loc[:, test.columns != "chd"] # 검증 데이터 중 독립 변수
y_test = test['chd'] # 검증 데이터 중 종속 변수

# 로지스틱 회귀모형 - 기본

In [22]:
# 기본 로지스틱 회귀모형 학습
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression(C=1e+10) 
# sklearn 의 Logistic Regression은 기본적으로 Ridge 정규화가 포함되어 있기 때문에, 
# 정규화 텀을 억제하는 C를 크게 적용한다 (C:Inverse of regularization strength)
lr_model.fit(x_train, y_train) # 로지스틱 회귀 모형 학습
lr_predict = lr_model.predict(x_test) # 학습 결과를 바탕으로 검증 데이터를 예측
lr_predict

array([0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0], dtype=int64)

In [23]:
# 기본 로지스틱 회귀모형 평가
from sklearn.metrics import confusion_matrix
lr_cm = confusion_matrix(y_test, lr_predict) # 실제 종속변수와, 예측한 종속변수를 바탕으로 Confusion Matrix
lr_cm

array([[62, 17],
       [16, 21]], dtype=int64)

In [24]:
# Accuracy, Recall, Precision, F1-Score
from sklearn.metrics import precision_recall_fscore_support
accuracy = (lr_cm[0][0] + lr_cm[1][1])/sum(sum(lr_cm))
precision, recall, fbeta_score, _ = precision_recall_fscore_support(y_test, lr_predict, average = 'binary')
print("Accuracy: %.2f" % accuracy)
print("Precision: %.2f" % precision)
print("Recall: %.2f" % recall)
print("F1 score: %.2f" % fbeta_score)

Accuracy: 0.72
Precision: 0.55
Recall: 0.57
F1 score: 0.56


# 로지스틱 회귀모형 - 라쏘

In [25]:
# 라쏘 로지스틱 회귀모형 학습
lasso_model = LogisticRegression(penalty='l1') # Penalty = l1 Regularizer, C = 1.0 (Default))
lasso_model.fit(x_train, y_train) # 라쏘 로지스틱 회귀 모형 학습
lasso_predict = lasso_model.predict(x_test) # 학습 결과를 바탕으로 검증 데이터를 예측
lasso_predict

array([0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0], dtype=int64)

In [26]:
# 라쏘 로지스틱 회귀모형 평가
lasso_cm = confusion_matrix(y_test, lasso_predict) # 실제 종속변수와, 예측한 종속변수를 바탕으로 Confusion Matrix
lasso_cm

array([[62, 17],
       [18, 19]], dtype=int64)

In [27]:
# Accuracy, Recall, Precision, F1-Score
accuracy = (lasso_cm[0][0] + lasso_cm[1][1])/sum(sum(lasso_cm))
precision, recall, fbeta_score, _ = precision_recall_fscore_support(y_test, lasso_predict, average = 'binary')
print("Accuracy: %.2f" % accuracy)
print("Precision: %.2f" % precision)
print("Recall: %.2f" % recall)
print("F1 score: %.2f" % fbeta_score)

Accuracy: 0.70
Precision: 0.53
Recall: 0.51
F1 score: 0.52


# 로지스틱 회귀모형 - 릿지

In [28]:
# 릿지 로지스틱 회귀모형 학습
ridge_model = LogisticRegression(penalty='l2') # Default = LogisticRegression()
ridge_model.fit(x_train, y_train) # 릿지 로지스틱 회귀 모형 학습
ridge_predict = ridge_model.predict(x_test) # 학습 결과를 바탕으로 검증 데이터를 예측
ridge_predict

array([0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 1], dtype=int64)

In [29]:
# 릿지 로지스틱 회귀모형 평가
ridge_cm = confusion_matrix(y_test, ridge_predict) # 실제 종속변수와, 예측한 종속변수를 바탕으로 Confusion Matrix
ridge_cm

array([[63, 16],
       [19, 18]], dtype=int64)

In [30]:
# Accuracy, Recall, Precision, F1-Score
accuracy = (ridge_cm[0][0] + ridge_cm[1][1])/sum(sum(ridge_cm))
precision, recall, fbeta_score, _ = precision_recall_fscore_support(y_test, ridge_predict, average = 'binary')
print("Accuracy: %.2f" % accuracy)
print("Precision: %.2f" % precision)
print("Recall: %.2f" % recall)
print("F1 score: %.2f" % fbeta_score)

Accuracy: 0.70
Precision: 0.53
Recall: 0.49
F1 score: 0.51
